In [1]:
import numpy as np
import tensorflow as tf
import logging
tf.get_logger().setLevel(logging.ERROR)

import utils

2025-04-27 19:25:42.915417: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-27 19:25:42.916699: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-27 19:25:42.940747: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-27 19:25:42.941160: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-27 19:25:43.251739: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

In [2]:
from mil.data.datasets import mnist_bags

In [3]:
# bags_train = np.load('train_dp_all_filt_features.npy')
# # bags_test = np.load('val_dp_all_filt_features.npy')
# bags_test = np.load('test_dp_all_filt_features.npy')


# y_train = np.load('train_dp_all_filt_labels.npy')
# # y_test = np.load('val_dp_all_filt_labels.npy')
# y_test = np.load('test_dp_all_filt_labels.npy')

# fpaths_train = np.load('train_dp_all_filt_files.npy')
# # fpaths_test = np.load('val_dp_all_filt_files.npy')
# fpaths_test = np.load('test_dp_all_filt_files.npy')

In [4]:
bags_train = np.load('sod4sb_train_features.npy')
bags_test = np.load('sod4sb_val_features.npy')

y_train = np.load('sod4sb_train_labels.npy')
y_test = np.load('sod4sb_val_labels.npy')

fpaths_train = np.load('sod4sb_train_files.npy')
fpaths_test = np.load('sod4sb_val_files.npy')

In [5]:
# bags_train = np.load('train_dp_filt_features.npy')
# bags_test = np.load('val_dp_filt_features.npy')

# y_train = np.load('train_dp_filt_labels.npy')
# y_test = np.load('val_dp_filt_labels.npy')

# fpaths_train = np.load('train_dp_filt_files.npy')
# fpaths_test = np.load('val_dp_filt_files.npy')

In [7]:
# maximum  number of instances in the training set
max_len_train = np.max([len(bag) for bag in bags_train])
max_len_test = np.max([len(bag) for bag in bags_test])

max_ = np.max([max_len_train, max_len_test])
max_

144

In [8]:
bags_train_1D = bags_train
bags_test_1D = bags_test

In [10]:
from mil.metrics import AUC, BinaryAccuracy, Sensibility, Specificity
from mil.validators import KFold
from mil.trainer.trainer import Trainer
from mil.models import SVC
from mil.bag_representation.mapping import DiscriminativeMapping
from mil.preprocessing import StandarizerBagsList, NormalizeBagsImage

from mil.models.bag_level.deep_attention import AttentionDeepPoolingMil
from mil.utils.utils import get_samples_weight
from mil.utils.padding import Padding

In [11]:
trainer = Trainer()

metrics = [AUC, BinaryAccuracy, Sensibility, Specificity]
model = AttentionDeepPoolingMil(gated=False, threshold=0.2)
pipeline = [('padding', Padding(max_len=max_))]

trainer.prepare(model, preprocess_pipeline=pipeline ,metrics=metrics)

2025-04-27 19:25:50.723854: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-04-27 19:25:50.749466: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [12]:
valid = KFold(n_splits=2, shuffle=True)

history = trainer.fit(bags_train, y_train, validation_strategy=valid, sample_weights='balanced',
                      verbose=1, model__epochs=20, model__batch_size=4, model__verbose=0)

2/2 [==============================] - 201s 99s/step - train_auc: 0.9741 - train_binaryaccuracy: 0.9236 - train_sensibility: 0.8765 - train_specificity: 0.9709 - val_auc: 0.9549 - val_binaryaccuracy: 0.9068 - val_sensibility: 0.8441 - val_specificity: 0.9743


In [15]:
print(np.mean([e['binaryaccuracy'] for e in history['metrics_val']]))
print(np.mean([e['auc'] for e in history['metrics_val']]))
history['metrics_val']

0.91451156
0.95961046


[{'auc': 0.9643493,
  'binaryaccuracy': 0.92218184,
  'sensibility': 0.90261626,
  'specificity': 0.94177586},
 {'auc': 0.9548716,
  'binaryaccuracy': 0.90684134,
  'sensibility': 0.84410113,
  'specificity': 0.97432023}]

In [16]:
trainer.predict_metrics(bags_test, y_test)

8/8 [==============================] - 0s 4ms/step


{'auc': 0.94389886,
 'binaryaccuracy': 0.9051724,
 'sensibility': 0.84166664,
 'specificity': 0.97321427}

In [17]:
# positive instances with more than 0.4 in attention weight
pos_test = trainer.get_positive_instances(bags_test)
pos_train = trainer.get_positive_instances(bags_train)

 1/86 [..............................] - ETA: 1s

/home/mcloud-ai/.newyolovenv310/lib/python3.10/site-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


86/86 [==============================] - 1s 7ms/step


In [21]:
import matplotlib.pyplot as plt
import cv2

In [ ]:
for img_index, patch_index in pos_test.numpy():
    fpath = fpaths_test[img_index]
    image = cv2.imread(fpath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    patches = utils.resize_and_crop(image, target_size=(1904, 1120), patch_size=224, half_patch=True)
    plt.imshow(patches[patch_index])
    plt.show()

In [ ]:
for img_index, patch_index in pos_train.numpy():
    fpath = fpaths_train[img_index]
    image = cv2.imread(fpath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    patches = utils.resize_and_crop(image, target_size=(1904, 1120), patch_size=224, half_patch=True)
    plt.imshow(patches[patch_index])
    plt.show()